# Names

- Brian Nguyen
- Nathan Wong
- Daniel Phan
- Ryley Mao

# Research Question

Can we build a predictive model to estimate how many points an NBA superstar will score in a given game, based on player performance metrics, game location, opponent defensive stats, and team offensive context? Specifically, we will use variables such as Points per Game (PPG), Field Goal Attempts per Game (FGA), Effective Field Goal Percentage (eFG%), True Shooting Percentage (TS%), home/away status, opponent defensive rating, opponent FG% allowed, opponent PPG allowed, team offensive rating, and team pace.

## Background and Prior Work

All members of our group are passionate NBA fans and active sports bettors, and we wanted to take a data-driven approach to improve that decision-making process. Millions of fans like us attempt to guess whether NBA superstars will score over a certain number of points on platforms like PrizePicks, often relying on intuition, matchup narratives, or raw stat trends. 

While many projects exist that aim to predict general fantasy performance or player stat lines, few focus specifically on individual point prediction for NBA superstars, and even fewer do so with an emphasis on practical, daily usability. For instance, a relevant GitHub project is "nba-player-points-prediction"<a name="cite_ref-1"></a>[<sup>1</sup>](#cite_note-1), which used Linear Regression to predict point totals but lacked player-specific model tailoring and did not incorporate matchup context or intangible factors like rest or home/away which are limitations we aim to avoid.

Another similar project, NBAplayer_Performance_Pred<a name="cite_ref-2"></a>[<sup>2</sup>](#cite_note-2), used Random Forest and Ridge Regression models to predict NBA player performance stats. While the project incorporated some useful statistical features, it focused more on general performance across all players rather than narrowing in on top scorers or real-world prediction usability. Also it did not include game-specific contextual variables such as rest days or injuries, which we believe are crucial for predicting scoring outputs on a game-to-game basis.

We want to design a tool that predicts a superstar’s point total on a given night by combining recent player performance, opponent defensive metrics, and game context (e.g., back-to-back games, home/away, rest days). Rather than building a generalized model across all players, we will narrow our scope to top scorers (top 50 in Points per game) in the regular season and aim for game-to-game predictive accuracy that could be realistically applied by fans, fantasy players, or bettors looking to gain an edge.	


1. <a name="cite_note-1"></a> [^](#cite_ref-1) https://github.com/Jayplect/nba-player-points-prediction
2. <a name="cite_note-2"></a> [^](#cite_ref-2) https://github.com/RayJiazy/NBAplayer_Performance_Pred

# Hypothesis


We hypothesize that a predictive model using player performance metrics, game location, opponent defensive stats, and team offensive context can accurately estimate how many points an NBA superstar will score in a given game. We expect that variables such as PPG, FGA, eFG%, TS%, home/away status, opponent defensive rating, opponent FG% allowed, opponent PPG allowed, team offensive rating, and team pace will show strong relationships with individual scoring. By capturing these factors, we believe the model will be able to make accurate point predictions within a reasonable margin. We believe this will be the outcome because superstar scoring is influenced by consistent patterns in usage, shooting efficiency, and team context, which are reflected in the metrics we selected.

# Data

## Data overview

For each dataset include the following information
- Dataset #1
  - Dataset Name: NBA Stats (1947-present)
  - Link to the dataset:https://www.kaggle.com/datasets/sumitrodatta/nba-aba-baa-stats/data
  - Number of observations: ~40000 
  - Number of variables: 7

This dataset contains NBA player and team statistics from 1947 to the present. For this project, we are focusing on the 2014–2025 seasons and 50 superstar players. Each observation represents a single player's performance in one game. Important variables in this dataset include Effective Field Goal Percentage (eFG%), True Shooting Percentage (TS%), opponent defensive rating, opponent field goal percentage allowed, opponent points per game allowed, team offensive rating, and team pace. These variables help us capture how efficiently players score and how difficult their matchups are.

- Dataset #2
  - Dataset Name: Complete NBA Database and Historical Box Scores
  - Link to the dataset:https://www.kaggle.com/datasets/eoinamoore/historical-nba-data-and-player-box-scores?select=PlayerStatistics.csv
  - Number of observations: ~40000 
  - Number of variables: 3

This second dataset includes game-by-game player stats for recent NBA seasons and provides the remaining key variables we need: Points Per Game (PPG), Field Goal Attempts (FGA), and game location (home/away). These variables are essential for measuring a player’s scoring output and how it might change depending on where the game is played.

This dataset contains NBA player and team statistics from the 1947 season through the present. For our project, we will focus on the 2014–2025 seasons and only include 50 superstar players. Each observation will represent one player's performance in a single game, leading to about 40,000 observations in total. Important variables include Points per Game (PPG), Field Goal Attempts (FGA), Effective Field Goal Percentage (eFG%), True Shooting Percentage (TS%), game location (home/away), opponent defensive rating, opponent field goal percentage allowed, opponent points per game allowed, team offensive rating, and team pace.

To build a complete dataset with all 10 variables, we will merge these two datasets using common keys such as player name, game date, and team. This will allow us to match each player's performance with the opponent and team context for that specific game. We need to filter our data so we are only looking at games from 2014-2025 and only regular season games. We also will be filtering for only our list of 50 superstars:

Nikola Jokic
Giannis Antetokounmpo
Shai Gilgeous Alexander
Luka Doncic
Victor Wembanyama
Anthony Davis
Cade Cunningham
Lebron James
Jayson Tatum
Karl-Anthony Towns
Domantas Sabonis
Kevin Durant
Trae Young
Zion Williamson 
Anthony Edwards
Damian Lillard
James Harden
Jalen Brunson 
Devin Booker
Tyrese Maxey
Paolo Banchero
Lamelo Ball
Stephen Curry
Joel Embiid
Kyrie Irving
Tyler Herro
De Aaron Fox
Jalen Johnson
Tyrese Haliburton
Franz Wagner
Ja Morant
Ivica Zubac
Alperen Sengun
Donovan Mitchell
Jalen Williams
Nikola Vucevic
Evan Mobley 
Jamal Murray
Scottie Barnes
Zach Lavine 
Brandon IIngram
Bam Adebayo
Jaylen Brown
Kawhi Leonard
Desmond Bane
Pascal Siakam
Darius Garland
RJ Barrett
Austin Reaves
Jaren Jackson Jr


## Setup

In [ ]:
import pandas as pd
from unidecode import unidecode

# Superstar players list
superstars = [
    "Nikola Jokic", "Giannis Antetokounmpo", "Shai Gilgeous Alexander", "Luka Doncic",
    "Victor Wembanyama", "Anthony Davis", "Cade Cunningham", "Lebron James", "Jayson Tatum",
    "Karl-Anthony Towns", "Domantas Sabonis", "Kevin Durant", "Trae Young", "Zion Williamson",
    "Anthony Edwards", "Damian Lillard", "James Harden", "Jalen Brunson", "Devin Booker",
    "Tyrese Maxey", "Paolo Banchero", "Lamelo Ball", "Stephen Curry", "Joel Embiid",
    "Kyrie Irving", "Tyler Herro", "De Aaron Fox", "Jalen Johnson", "Tyrese Haliburton",
    "Franz Wagner", "Ja Morant", "Ivica Zubac", "Alperen Sengun", "Donovan Mitchell",
    "Jalen Williams", "Nikola Vucevic", "Evan Mobley", "Jamal Murray", "Scottie Barnes",
    "Zach Lavine", "Brandon IIngram", "Bam Adebayo", "Jaylen Brown", "Kawhi Leonard",
    "Desmond Bane", "Pascal Siakam", "Darius Garland", "RJ Barrett", "Austin Reaves",
    "Jaren Jackson Jr"
]
superstars = [unidecode(name.lower().strip()) for name in superstars]

## Dataset #1 (use name instead of number here)

In [77]:
# Team summaries
team_summary = pd.read_csv('data/Team Summaries.csv')
team_summary['team'] = team_summary['team'].str.split().str[-1]
team_summary = team_summary[['season','team','o_rtg','d_rtg', 'pace']]
team_summary = team_summary[(team_summary['season'] >= 2014) & (team_summary['season'] <= 2025)]

# Opponent stats
opp_stats = pd.read_csv('data/Opponent Stats Per Game.csv')
opp_stats['team'] = opp_stats['team'].str.split().str[-1]
opp_stats = opp_stats[['season', 'team', 'opp_fg_percent', 'opp_pts_per_game']]
opp_stats = opp_stats[(opp_stats['season'] >= 2014) & (opp_stats['season'] <= 2025)]

# Advanced stats
adv = pd.read_csv('data/Advanced.csv')
adv = adv[['season', 'player', 'ts_percent']]
adv = adv[(adv['season'] >= 2014) & (adv['season'] <= 2025)]
adv['player'] = adv['player'].apply(lambda x: unidecode(x.strip().lower()))
adv = adv[adv['player'].isin(superstars)]
adv = adv.groupby(['season', 'player'], as_index=False).agg({'ts_percent': 'mean'})

# Per game stats
game = pd.read_csv('data/Player Per Game.csv')
game = game[['season', 'player', 'e_fg_percent']]
game = game[(game['season'] >= 2014) & (game['season'] <= 2025)]
game['player'] = game['player'].apply(lambda x: unidecode(x.strip().lower()))
game = game[game['player'].isin(superstars)]
game = game.groupby(['season','player'], as_index=False).agg({'e_fg_percent':'mean'})



## Dataset #2 (use name instead of number here)

In [78]:
# Player game-by-game
stats = pd.read_csv('data/PlayerStatistics.csv')
stats['player'] = stats['firstName'] + ' ' + stats['lastName']
stats['player'] = stats['player'].apply(lambda x: unidecode(x.strip().lower()))
stats['gameDate'] = pd.to_datetime(stats['gameDate'])
stats['season'] = stats['gameDate'].apply(lambda x: x.year + 1 if x.month >= 10 else x.year)
stats = stats[(stats['gameDate'].dt.year >= 2014) & (stats['gameDate'].dt.year <= 2025)]
stats = stats[stats['player'].isin(superstars)]
stats = stats[stats['gameType'] == 'Regular Season']
stats = stats[['player', 'season', 'playerteamName', 'opponentteamName', 'home', 'points', 'fieldGoalsAttempted']]
stats['team'] = stats['playerteamName'].str.split().str[-1]

C:\Users\Brian\AppData\Local\Temp\ipykernel_1660\616632331.py:2: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  stats = pd.read_csv('data/PlayerStatistics.csv')


## Merge into one Data Frame

In [ ]:
# Merge team summaries + opponent stats
team_stats = pd.merge(team_summary, opp_stats, on=['season', 'team'], how='inner')
team_stats = team_stats.sort_values(by='season')

# Merge advanced stats + per game stats
player_stats = pd.merge(adv, game, on=['season', 'player'], how='inner')

# Merge with team stats
df_merged = stats.merge(team_stats, on=['season', 'team'], how='left')

# Final merge with player stats
df = df_merged.merge(player_stats, on=['season', 'player'], how='left')

print(df)


                player  season playerteamName opponentteamName  home  points  \
0         nikola jokic    2025        Nuggets          Rockets     0    18.0   
1         jamal murray    2025        Nuggets          Rockets     0    16.0   
2       alperen sengun    2025        Rockets          Nuggets     1    14.0   
3      anthony edwards    2025   Timberwolves             Jazz     1    43.0   
4           rj barrett    2025        Raptors            Spurs     0     0.0   
...                ...     ...            ...              ...   ...     ...   
22829    kawhi leonard    2014          Spurs           Knicks     1     8.0   
22830     kevin durant    2014        Thunder             Nets     1    24.0   
22831     lebron james    2014           Heat         Warriors     1    26.0   
22832    stephen curry    2014       Warriors             Heat     0    36.0   
22833    anthony davis    2014       Pelicans     Timberwolves     0    13.0   

       fieldGoalsAttempted          tea

## Check Null Values and Data Types

In [81]:
print(df.isnull().sum())
print(df.dtypes)

player                 0
season                 0
playerteamName         0
opponentteamName       0
home                   0
points                 0
fieldGoalsAttempted    0
team                   0
o_rtg                  0
d_rtg                  0
pace                   0
opp_fg_percent         0
opp_pts_per_game       0
ts_percent             0
e_fg_percent           0
dtype: int64
player                  object
season                   int64
playerteamName          object
opponentteamName        object
home                     int64
points                 float64
fieldGoalsAttempted    float64
team                    object
o_rtg                  float64
d_rtg                  float64
pace                   float64
opp_fg_percent         float64
opp_pts_per_game       float64
ts_percent             float64
e_fg_percent           float64
dtype: object


# Ethics & Privacy

There should not be significant privacy or ethical concerns with the data we are collecting, as it comes from public sources like the NBA’s official statistics website, Basketball-Reference, and APIs such as nba_api. These sources are reputable and widely used for research and fan analysis. Since player statistics, team metrics, and game schedules are already publicly available, we are not using any private or personally identifiable information. However, one potential ethical concern is the use of injury information. While injury status is often publicly reported, we will treat it cautiously and avoid making assumptions about a player's health or long-term performance based on brief injury labels. We will only use high-level injury status (e.g., whether the player played or not) and not speculate beyond that.

There is also a potential bias in who is included in our dataset, as we are only focusing on the top 50 scorers in the league. This could skew our model toward players with high offensive usage and media exposure, possibly ignoring players from smaller-market teams or more defensive-focused roles. While this limitation is acceptable for our specific research question, which targets high-scoring players, we acknowledge that it may limit the generalizability of our model. Additionally, factors such as national TV coverage, travel schedules, or coaching decisions may introduce unseen biases that impact player performance but are difficult to quantify.

Another limitation is that our model may not fully account for game context such as crowd influence, altitude, or locker room issues—factors that are hard to measure but can influence performance. We will also be cautious about interpreting our model’s predictions as exact truths; instead, we will present them as probabilistic estimates with expected variance. Since our project may appeal to fantasy sports or betting audiences, we will include a clear disclaimer that our model is for educational use only and should not be taken as financial advice.

To address these concerns, we will review our dataset for coverage gaps (e.g., missing games, skewed representation of certain teams or players), compare prediction accuracy across player types, and document any trends or outliers. During analysis, we will flag inconsistencies or anomalies and conduct additional research to decide whether they reflect natural variance or underlying bias. Finally, before finalizing our results, we will proofread our conclusions with an eye toward ethical communication and may ask for outside feedback to catch unintentional bias in our interpretations.

# Team Expectations 

* *Keep in touch with group members via iMessage or Discord groupchat*
* *Coordinate on meeting times*
* *Complete individually assigned tasks on time*
* *Communicate promptly if any change of plans arises*

# Project Timeline Proposal

Specify your team's specific project timeline. An example timeline has been provided. Changes the dates, times, names, and details to fit your group's plan.

If you think you will need any special resources or training outside what we have covered in COGS 108 to solve your problem, then your proposal should state these clearly. For example, if you have selected a problem that involves implementing multiple neural networks, please state this so we can make sure you know what you’re doing and so we can point you to resources you will need to implement your project. Note that you are not required to use outside methods.



| Meeting Date  | Meeting Time| Completed Before Meeting  | Discuss at Meeting |
|---|---|---|---|
| 4/30  |  4 PM | Brainstorm project ideas  | Finish Project Proposal | 
| 5/7 |  4 PM |  Do background research on topic and find Data | Work on Checkpoint #1: Data | 
| 5/14  | 4 PM  | Import & Wrangle Data  | Finish Checkpoint #1: Data   |
| 5/21 | 4 PM  | Work on Assigned Tasks | Review/Edit wrangling/EDA; Discuss Analysis Plan ; Start Checkpoint #2: EDA  |
| 5/28  | 4 PM  | Finalize wrangling/EDA; Begin Analysis | Finish Checkpoint #2: EDA |
| 6/4  | 4 PM  | Complete analysis; Draft results/conclusion/discussion | Discuss/edit full project |
| 6/13  | 4 PM  | Make final edits and prepare to turn in Final Project | Turn in Final Project & Group Project Surveys |